## Calculate mean and standard deviation of azimuth results for each Grade AA and BB event

### Import libraries

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os
import glob
import sys
from scipy import stats

### Load catalog of high quality events

In [2]:
mqdir1 = '../catalogs/final_catalogs/'
cat = pd.read_csv(mqdir1 + 'A17_moonquakes_catalog_HQ_final.csv')
cat.drop(list(cat.filter(regex='Unnamed|index|mean_cc')), axis=1, inplace=True)

### Load catalogs of azimuth results

In [3]:
# Lag
mqdir2 = '../catalogs/azimuths/'
lag_df = pd.read_csv(mqdir2 + 'A17_moonquakes_AA_BB_azimuths_lag.csv')

# SNR function
snr_df = pd.read_csv(mqdir2 + 'A17_moonquakes_AA_BB_azimuths_SNR.csv')

### Iterate through each event and calculate mean and SD of azimuth results

In [13]:
mean_azimuths_lag = []
sd_azimuths_lag = []
mean_azimuths_snr = []
sd_azimuths_snr = []

# Iterate through rows of moonquake
for r in np.arange(0,len(cat)):
    
    # Check for azimuth results
    row = cat.iloc[r]
    evid = row.evid
    rows_lag = lag_df.loc[lag_df.evid == evid]
    rows_snr = snr_df.loc[snr_df.evid == evid]
    
    # Results don't exist
    if len(rows_lag) == 0:
        mean_azimuths_lag.append(-1)
        sd_azimuths_lag.append(-1)
        mean_azimuths_snr.append(-1)
        sd_azimuths_snr.append(-1)
    
    # Results exist
    else:
        # Azimuths of each iteration
        thetas_lag = np.array(rows_lag.theta_deg.tolist())
        thetas_snr = np.array(rows_snr.theta_deg.tolist())
        
        # Calculate circular mean and standard deviation following Mardia and Jupp (1999)
        # Mardia, K.V. & Jupp, P.E., 1999. Circular Data. in Directional Statistics, 
        # pp. 1–12, John Wiley & Sons, Ltd. doi:10.1002/9780470316979.ch1
        mean_lag = np.rad2deg(stats.circmean(np.deg2rad(thetas_lag)))
        sd_lag = np.rad2deg(stats.circstd(np.deg2rad(thetas_lag)))
        mean_snr = np.rad2deg(stats.circmean(np.deg2rad(thetas_snr)))
        sd_snr = np.rad2deg(stats.circstd(np.deg2rad(thetas_snr)))
        
        # Append
        mean_azimuths_lag.append(mean_lag)
        sd_azimuths_lag.append(sd_lag)
        mean_azimuths_snr.append(mean_snr)
        sd_azimuths_snr.append(sd_snr)

### Add to catalog and save

In [14]:
cat['theta_mean_lag_deg'] = mean_azimuths_lag
cat['theta_sd_lag_deg'] = sd_azimuths_lag
cat['theta_mean_snr_deg'] = mean_azimuths_snr
cat['theta_sd_snr_deg'] = sd_azimuths_snr

In [15]:
cat.to_csv(mqdir1 + 'A17_moonquakes_catalog_HQ_final.csv',index=False)